# LangGraph MCP Tool-Calling Agent - Interactive Testing

This notebook demonstrates how to test and deploy a LangGraph agent that connects to MCP servers hosted on Databricks.

## Features
- Author a LangGraph agent wrapped with `ResponsesAgent` for Mosaic AI compatibility
- Call MCP tools (managed or custom)
- Manually test the agent
- Evaluate with Mosaic AI Agent Evaluation
- Log and deploy the agent

In [1]:
# COMMAND ----------

# Install required packages
%pip install -U -qqqq langgraph uv databricks-agents mlflow-skinny[databricks] databricks-mcp databricks-langchain azure-identity azure-storage-blob azure-core

zsh:1: no matches found: mlflow-skinny[databricks]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# COMMAND ----------

## Step 1: Define the Agent Code

Write the agent code to a local Python file using the `%%writefile` magic command.

# COMMAND ----------

## Step 0: Local Authentication Setup (for running outside Databricks)

When running this notebook locally, you need to authenticate with Databricks. You have two options:

### Option 1: Databricks CLI OAuth Login (Recommended for local development)
```bash
# Install Databricks CLI if not already installed
pip install databricks-cli

# Login using OAuth (opens browser for authentication)
databricks auth login --host https://your-workspace.cloud.databricks.com
```

This creates a configuration profile that the SDK will automatically use.

### Option 2: Environment Variables with OAuth M2M
For automated/CI environments, use OAuth machine-to-machine with a service principal:
```bash
export DATABRICKS_HOST="https://your-workspace.cloud.databricks.com"
export DATABRICKS_CLIENT_ID="your-service-principal-client-id"
export DATABRICKS_CLIENT_SECRET="your-service-principal-secret"
```

The code below will automatically detect and use whichever authentication method is available.

In [2]:
# COMMAND ----------

# Test Databricks authentication
from databricks.sdk import WorkspaceClient
import os

# Use the profile name (for local development)
profile_name = os.getenv("DATABRICKS_CONFIG_PROFILE", "development")

try:
    ws = WorkspaceClient(profile=profile_name)
    print(f"✓ Successfully authenticated to: {ws.config.host}")
    print(f"✓ Using profile: {profile_name}")
    print(f"✓ Auth type: {ws.config.auth_type}")
except Exception as e:
    print(f"✗ Authentication failed: {e}")
    print("\nTo fix, run: databricks auth login --host https://your-workspace.cloud.databricks.com")

✓ Successfully authenticated to: https://adb-3253299566947192.12.azuredatabricks.net
✓ Using profile: development
✓ Auth type: metadata-service


In [3]:
%%writefile agent.py
import asyncio
from typing import Annotated, Any, Generator, List, Optional, Sequence, TypedDict, Union

import mlflow
import nest_asyncio
from databricks.sdk import WorkspaceClient
from databricks_langchain import (
    ChatDatabricks,
    UCFunctionToolkit,
    VectorSearchRetrieverTool,
)
from databricks_mcp import DatabricksMCPClient, DatabricksOAuthClientProvider
from langchain.messages import AIMessage, AIMessageChunk, AnyMessage
from langchain_core.language_models import LanguageModelLike
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.tools import BaseTool
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_node import ToolNode
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client as connect
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
    output_to_responses_items_stream,
    to_chat_completions_input,
)
from pydantic import create_model

nest_asyncio.apply()

############################################
## Define your LLM endpoint and system prompt
############################################
# TODO: Replace with your model serving endpoint
LLM_ENDPOINT_NAME = "databricks-claude-3-7-sonnet"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

# TODO: Update with your system prompt
system_prompt = "You are a helpful assistant that can run Python code."

###############################################################################
## Configure MCP Servers for your agent
##
## This section sets up server connections so your agent can retrieve data or take actions.

## There are three connection types:
## 1. Managed MCP servers — fully managed by Databricks (no setup required)
## 2. External MCP servers — hosted outside Databricks but proxied through a
##    Managed MCP server proxy (some setup required)
## 3. Custom MCP servers — MCP servers hosted as Databricks Apps (OAuth setup required)
##
## Note: External MCP servers get added to the "managed" URL list
## because their proxy endpoints are managed by Databricks.
###############################################################################

# TODO: Choose your MCP server connection type and fill in the appropriate URLs.

# ---------------------------------------------------------------------------
# Managed MCP Server — simplest setup
# ---------------------------------------------------------------------------
# Databricks manages this connection automatically using your workspace settings.
# When running locally, it will use authentication from:
#   1. Databricks CLI OAuth profiles (recommended - run: databricks auth login)
#   2. Environment variables (DATABRICKS_HOST, DATABRICKS_CLIENT_ID, DATABRICKS_CLIENT_SECRET)
#   3. Default profile from ~/.databrickscfg

import os

# For local development, specify the profile name if you have multiple profiles
# Comment this out when running in Databricks notebooks
profile_name = os.getenv("DATABRICKS_CONFIG_PROFILE", "development")  # Use your profile name

try:
    # Try to use the specified profile (for local development)
    workspace_client = WorkspaceClient(profile=profile_name)
except Exception:
    # Fall back to default authentication (works in Databricks notebooks)
    workspace_client = WorkspaceClient()

host = workspace_client.config.host

# Managed MCP Servers URLS (includes both fully managed and proxied external MCP)
# - If you're using an external MCP server, create a UC connection and flag it
#   as an MCP connection. This reveals a proxy endpoint.
# - Add that proxy endpoint URL to this list.

MANAGED_MCP_SERVER_URLS = [
    f"{host}/api/2.0/mcp/functions/system/ai",  # Default managed MCP endpoint
    # Example for external MCP:
    # "https://<workspace-hostname>/api/2.0/mcp/external/{connection_name}"
]

# ---------------------------------------------------------------------------
# Custom MCP Server — hosted as a Databricks App
# ---------------------------------------------------------------------------
# Use this if you're running your own MCP server in Databricks.
# These require OAuth with a service principal for machine-to-machine (M2M) auth.
#
# Uncomment and fill in the settings below to use a custom MCP server.
#
# import os
# workspace_client = WorkspaceClient(
#     host="<DATABRICKS_WORKSPACE_URL>",
#     client_id=os.getenv("DATABRICKS_CLIENT_ID"),
#     client_secret=os.getenv("DATABRICKS_CLIENT_SECRET"),
#     auth_type="oauth-m2m",  # Enables service principal authentication
# )

# Custom MCP Servers — add URLs below (not managed or proxied by Databricks)
CUSTOM_MCP_SERVER_URLS = [
    # Example: "https://<custom-mcp-app-url>/mcp"
]


#####################
## MCP Tool Creation
#####################

# Define a custom LangChain tool that wraps functionality for calling MCP servers
class MCPTool(BaseTool):
    """Custom LangChain tool that wraps MCP server functionality"""

    def __init__(
        self,
        name: str,
        description: str,
        args_schema: type,
        server_url: str,
        ws: WorkspaceClient,
        is_custom: bool = False,
    ):
        # Initialize the tool
        super().__init__(name=name, description=description, args_schema=args_schema)
        # Store custom attributes: MCP server URL, Databricks workspace client, and whether the tool is for a custom server
        object.__setattr__(self, "server_url", server_url)
        object.__setattr__(self, "workspace_client", ws)
        object.__setattr__(self, "is_custom", is_custom)

    def _run(self, **kwargs) -> str:
        """Execute the MCP tool"""
        if self.is_custom:
            # Use the async method for custom MCP servers (OAuth required)
            return asyncio.run(self._run_custom_async(**kwargs))
        else:
            # Use managed MCP server via synchronous call
            mcp_client = DatabricksMCPClient(
                server_url=self.server_url, workspace_client=self.workspace_client
            )
            response = mcp_client.call_tool(self.name, kwargs)
            return "".join([c.text for c in response.content])

    async def _run_custom_async(self, **kwargs) -> str:
        """Execute custom MCP tool asynchronously"""
        async with connect(
            self.server_url, auth=DatabricksOAuthClientProvider(self.workspace_client)
        ) as (
            read_stream,
            write_stream,
            _,
        ):
            # Create an async session with the server and call the tool
            async with ClientSession(read_stream, write_stream) as session:
                await session.initialize()
                response = await session.call_tool(self.name, kwargs)
                return "".join([c.text for c in response.content])


# Retrieve tool definitions from a custom MCP server (OAuth required)
async def get_custom_mcp_tools(ws: WorkspaceClient, server_url: str):
    """Get tools from a custom MCP server using OAuth"""
    async with connect(server_url, auth=DatabricksOAuthClientProvider(ws)) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tools_response = await session.list_tools()
            return tools_response.tools


# Retrieve tool definitions from a managed MCP server
def get_managed_mcp_tools(ws: WorkspaceClient, server_url: str):
    """Get tools from a managed MCP server"""
    mcp_client = DatabricksMCPClient(server_url=server_url, workspace_client=ws)
    return mcp_client.list_tools()


# Convert an MCP tool definition into a LangChain-compatible tool
def create_langchain_tool_from_mcp(
    mcp_tool, server_url: str, ws: WorkspaceClient, is_custom: bool = False
):
    """Create a LangChain tool from an MCP tool definition"""
    schema = mcp_tool.inputSchema.copy()
    properties = schema.get("properties", {})
    required = schema.get("required", [])

    # Map JSON schema types to Python types for input validation
    TYPE_MAPPING = {"integer": int, "number": float, "boolean": bool}
    field_definitions = {}
    for field_name, field_info in properties.items():
        field_type_str = field_info.get("type", "string")
        field_type = TYPE_MAPPING.get(field_type_str, str)

        if field_name in required:
            field_definitions[field_name] = (field_type, ...)
        else:
            field_definitions[field_name] = (field_type, None)

    # Dynamically create a Pydantic schema for the tool's input arguments
    args_schema = create_model(f"{mcp_tool.name}Args", **field_definitions)

    # Return a configured MCPTool instance
    return MCPTool(
        name=mcp_tool.name,
        description=mcp_tool.description or f"Tool: {mcp_tool.name}",
        args_schema=args_schema,
        server_url=server_url,
        ws=ws,
        is_custom=is_custom,
    )


# Gather all tools from managed and custom MCP servers into a single list
async def create_mcp_tools(
    ws: WorkspaceClient, managed_server_urls: List[str] = None, custom_server_urls: List[str] = None
) -> List[MCPTool]:
    """Create LangChain tools from both managed and custom MCP servers"""
    tools = []

    if managed_server_urls:
        # Load managed MCP tools
        for server_url in managed_server_urls:
            try:
                mcp_tools = get_managed_mcp_tools(ws, server_url)
                for mcp_tool in mcp_tools:
                    tool = create_langchain_tool_from_mcp(mcp_tool, server_url, ws, is_custom=False)
                    tools.append(tool)
            except Exception as e:
                print(f"Error loading tools from managed server {server_url}: {e}")

    if custom_server_urls:
        # Load custom MCP tools (async)
        for server_url in custom_server_urls:
            try:
                mcp_tools = await get_custom_mcp_tools(ws, server_url)
                for mcp_tool in mcp_tools:
                    tool = create_langchain_tool_from_mcp(mcp_tool, server_url, ws, is_custom=True)
                    tools.append(tool)
            except Exception as e:
                print(f"Error loading tools from custom server {server_url}: {e}")

    return tools


#####################
## Define agent logic
#####################


# The state for the agent workflow, including the conversation and any custom data
class AgentState(TypedDict):
    messages: Annotated[Sequence[AnyMessage], add_messages]
    custom_inputs: Optional[dict[str, Any]]
    custom_outputs: Optional[dict[str, Any]]


# Define the LangGraph agent that can call tools
def create_tool_calling_agent(
    model: LanguageModelLike,
    tools: Union[ToolNode, Sequence[BaseTool]],
    system_prompt: Optional[str] = None,
):
    model = model.bind_tools(tools)  # Bind tools to the model

    # Function to check if agent should continue or finish based on last message
    def should_continue(state: AgentState):
        messages = state["messages"]
        last_message = messages[-1]
        # If function (tool) calls are present, continue; otherwise, end
        if isinstance(last_message, AIMessage) and last_message.tool_calls:
            return "continue"
        else:
            return "end"

    # Preprocess: optionally prepend a system prompt to the conversation history
    if system_prompt:
        preprocessor = RunnableLambda(
            lambda state: [{"role": "system", "content": system_prompt}] + state["messages"]
        )
    else:
        preprocessor = RunnableLambda(lambda state: state["messages"])

    model_runnable = preprocessor | model  # Chain the preprocessor and the model

    # The function to invoke the model within the workflow
    def call_model(
        state: AgentState,
        config: RunnableConfig,
    ):
        response = model_runnable.invoke(state, config)
        return {"messages": [response]}

    workflow = StateGraph(AgentState)  # Create the agent's state machine

    workflow.add_node("agent", RunnableLambda(call_model))  # Agent node (LLM)
    workflow.add_node("tools", ToolNode(tools))  # Tools node

    workflow.set_entry_point("agent")  # Start at agent node
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "continue": "tools",  # If the model requests a tool call, move to tools node
            "end": END,  # Otherwise, end the workflow
        },
    )
    workflow.add_edge("tools", "agent")  # After tools are called, return to agent node

    # Compile and return the tool-calling agent workflow
    return workflow.compile()


# ResponsesAgent class to wrap the compiled agent and make it compatible with Mosaic AI Responses API
class LangGraphResponsesAgent(ResponsesAgent):
    def __init__(self, agent):
        self.agent = agent

    # Make a prediction (single-step) for the agent
    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done" or event.type == "error"
        ]
        return ResponsesAgentResponse(output=outputs, custom_outputs=request.custom_inputs)

    # Stream predictions for the agent, yielding output as it's generated
    def predict_stream(
        self,
        request: ResponsesAgentRequest,
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        cc_msgs = to_chat_completions_input([i.model_dump() for i in request.input])
        # Stream events from the agent graph
        for event in self.agent.stream({"messages": cc_msgs}, stream_mode=["updates", "messages"]):
            if event[0] == "updates":
                # Stream updated messages from the workflow nodes
                for node_data in event[1].values():
                    if len(node_data.get("messages", [])) > 0:
                        yield from output_to_responses_items_stream(node_data["messages"])
            elif event[0] == "messages":
                # Stream generated text message chunks
                try:
                    chunk = event[1][0]
                    if isinstance(chunk, AIMessageChunk) and (content := chunk.content):
                        yield ResponsesAgentStreamEvent(
                            **self.create_text_delta(delta=content, item_id=chunk.id),
                        )
                except:
                    pass


# Initialize the entire agent, including MCP tools and workflow
def initialize_agent():
    """Initialize the agent with MCP tools"""
    # Create MCP tools from the configured servers
    mcp_tools = asyncio.run(
        create_mcp_tools(
            ws=workspace_client,
            managed_server_urls=MANAGED_MCP_SERVER_URLS,
            custom_server_urls=CUSTOM_MCP_SERVER_URLS,
        )
    )

    # Create the agent graph with an LLM, tool set, and system prompt (if given)
    agent = create_tool_calling_agent(llm, mcp_tools, system_prompt)
    return LangGraphResponsesAgent(agent)


# Configure MLflow for Databricks deployment (optional for local testing)
# Only enable if you want to track runs to Databricks workspace
def setup_mlflow():
    """Setup MLflow tracking and model registration"""
    try:
        mlflow.langchain.autolog()
        print("✓ MLflow autologging enabled")
    except Exception as e:
        print(f"Warning: MLflow autologging failed: {e}")

# Initialize agent
AGENT = initialize_agent()

# Try to set up MLflow model tracking (optional)
try:
    setup_mlflow()
    mlflow.models.set_model(AGENT)
except Exception as e:
    print(f"Note: MLflow model tracking not available: {e}")
    print("Agent will work without MLflow tracking.")

Overwriting agent.py


# COMMAND ----------

## Step 2: Test the Agent

Load and test the agent with sample queries.

In [4]:
# COMMAND ----------

# Configure MLflow tracking BEFORE importing agent
import mlflow
import os

profile_name = os.getenv("DATABRICKS_CONFIG_PROFILE", "development")

try:
    # Set tracking URI with profile
    mlflow.set_tracking_uri(f"databricks://{profile_name}")
    print(f"✓ MLflow tracking configured: databricks://{profile_name}")
    
    # Set experiment name
    experiment_name = "/Users/huy.d@hotmail.com/langgraph-mcp-agent"  # TODO: Update with your email
    mlflow.set_experiment(experiment_name)
    print(f"✓ Experiment set to: {experiment_name}")
except Exception as e:
    print(f"Note: MLflow tracking not configured: {e}")
    print("Agent will work without MLflow tracking.")

✓ MLflow tracking configured: databricks://development
✓ Experiment set to: /Users/huy.d@hotmail.com/langgraph-mcp-agent
✓ Experiment set to: /Users/huy.d@hotmail.com/langgraph-mcp-agent


In [5]:
# COMMAND ----------

from agent import AGENT

# Test the agent with a simple query
response = AGENT.predict({
    "input": [
        {"role": "user", "content": "What is 7*6 in Python?"}
    ]
})

print(response)

✓ MLflow autologging enabled
tool_choice=None truncation=None id=None created_at=None error=None incomplete_details=None instructions=None metadata=None model=None object='response' output=[OutputItem(type='message', id='lc_run--57685e74-4a60-4850-840a-7a399f7fb2c8', content=[{'text': 'I can calculate 7*6 using Python. Let me run that for you:', 'type': 'output_text'}], role='assistant'), OutputItem(type='function_call', id='lc_run--57685e74-4a60-4850-840a-7a399f7fb2c8', call_id='toolu_bdrk_01RtVY3SpWi14fWqPDcVFefM', name='system__ai__python_exec', arguments='{"code": "print(7 * 6)"}'), OutputItem(type='function_call_output', call_id='toolu_bdrk_01RtVY3SpWi14fWqPDcVFefM', output='{"is_truncated":false,"columns":["output"],"rows":[["42\\n"]]}'), OutputItem(type='message', id='lc_run--dd34d21c-ea34-4c6d-b0c2-a20724ddd01b', content=[{'text': 'The result of 7 * 6 in Python is 42.', 'type': 'output_text'}], role='assistant')] parallel_tool_calls=None temperature=None tools=None top_p=None m

In [6]:
# COMMAND ----------

print(response)
for chunk in AGENT.predict_stream({
    "input": [
        {"role": "user", "content": "Calculate the 15th Fibonacci number using Python"}
    ]
}):
    print(chunk, "-----------\n")

tool_choice=None truncation=None id=None created_at=None error=None incomplete_details=None instructions=None metadata=None model=None object='response' output=[OutputItem(type='message', id='lc_run--57685e74-4a60-4850-840a-7a399f7fb2c8', content=[{'text': 'I can calculate 7*6 using Python. Let me run that for you:', 'type': 'output_text'}], role='assistant'), OutputItem(type='function_call', id='lc_run--57685e74-4a60-4850-840a-7a399f7fb2c8', call_id='toolu_bdrk_01RtVY3SpWi14fWqPDcVFefM', name='system__ai__python_exec', arguments='{"code": "print(7 * 6)"}'), OutputItem(type='function_call_output', call_id='toolu_bdrk_01RtVY3SpWi14fWqPDcVFefM', output='{"is_truncated":false,"columns":["output"],"rows":[["42\\n"]]}'), OutputItem(type='message', id='lc_run--dd34d21c-ea34-4c6d-b0c2-a20724ddd01b', content=[{'text': 'The result of 7 * 6 in Python is 42.', 'type': 'output_text'}], role='assistant')] parallel_tool_calls=None temperature=None tools=None top_p=None max_output_tokens=None previou

# COMMAND ----------

## Step 3: Evaluate the Agent

Run evaluation with predefined LLM scorers.

In [7]:
# COMMAND ----------

import mlflow
from mlflow.genai.scorers import RelevanceToQuery, Safety

eval_dataset = [
    {
        "inputs": {
            "input": [
                {
                    "role": "user",
                    "content": "Calculate the 15th Fibonacci number"
                }
            ]
        },
        "expected_response": "The 15th Fibonacci number is 610."
    }
]

eval_results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=lambda input: AGENT.predict({"input": input}),
    scorers=[RelevanceToQuery(), Safety()],
)

# Review the evaluation results in the MLflow UI
print("Evaluation complete. Check the MLflow UI for detailed results.")

2025/11/08 22:44:33 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2025/11/08 22:44:33 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset. To disable this check, set the MLFLOW_GENAI_EVAL_SKIP_TRACE_VALIDATION environment variable to True.
2025/11/08 22:44:33 WARNING mlflow.tracing.fluent: Failed to start span predict_stream: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.
2025/11/08 22:44:33 WARNING mlflow.tracing.fluent: Failed to start span LangGraph: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.
2025/11/08 22:44:33 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset. To disable this check, set the MLFLOW_GENAI_EVAL_S

Evaluation complete. Check the MLflow UI for detailed results.


# COMMAND ----------

## Step 4: Log the Agent as an MLflow Model

Log the agent code for deployment.

In [8]:
# COMMAND ----------

import mlflow
import os
from agent import LLM_ENDPOINT_NAME
from mlflow.models.resources import DatabricksServingEndpoint, DatabricksFunction
from pkg_resources import get_distribution

# Set tracking URI to Databricks workspace with profile
# MLflow supports 'databricks://{profile}' format for multiple profiles
profile_name = os.getenv("DATABRICKS_CONFIG_PROFILE", "development")

try:
    # Use profile-specific tracking URI
    mlflow.set_tracking_uri(f"databricks://{profile_name}")
    print(f"✓ MLflow tracking configured with profile: {profile_name}")
    
    # Set experiment name
    experiment_name = "/Users/huy.d@hotmail.com/langgraph-mcp-agent"  # TODO: Update with your email
    mlflow.set_experiment(experiment_name)
    print(f"✓ Experiment set to: {experiment_name}")
except Exception as e:
    print(f"Warning: Could not set tracking URI: {e}")
    print("Continuing with default tracking URI...")

resources = [
    DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME), 
    DatabricksFunction(function_name="system.ai.python_exec")
]

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        resources=resources,
        pip_requirements=[
            "databricks-mcp",
            f"langgraph=={get_distribution('langgraph').version}",
            f"mcp=={get_distribution('mcp').version}",
            f"databricks-langchain=={get_distribution('databricks-langchain').version}",
        ]
    )

print(f"Model logged: {logged_agent_info.model_uri}")

✓ MLflow tracking configured with profile: development


/var/folders/gg/dkwbth_n51q5568p928r8r_m0000gn/T/ipykernel_82190/1277033277.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution


✓ Experiment set to: /Users/huy.d@hotmail.com/langgraph-mcp-agent


🔗 View Logged Model at: https://adb-3253299566947192.12.azuredatabricks.net/ml/experiments/897801252548174/models/m-917a987360744911bce572bc7e4c090c
2025/11/08 22:44:55 INFO mlflow.pyfunc: Predicting on input example to validate output
2025/11/08 22:44:55 WARNING mlflow.tracing.fluent: Failed to start span predict_stream: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.
2025/11/08 22:44:55 WARNING mlflow.tracing.fluent: Failed to start span LangGraph: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.
2025/11/08 22:44:55 INFO mlflow.pyfunc: Predicting on input example to validate output
2025/11/08 22:44:55 WARNING mlflow.tracing.fluent: Failed to start span predict_stream: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.
2025/11/08 22:44:55 WARNING mlflow.tracing.fluent: Failed to start span LangGraph: 'NonRecordingSpan' object has

✓ MLflow autologging enabled


2025/11/08 22:44:58 WARNING mlflow.tracing.fluent: Failed to start span predict_stream: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.
2025/11/08 22:44:58 WARNING mlflow.tracing.fluent: Failed to start span LangGraph: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.
2025/11/08 22:44:58 WARNING mlflow.tracing.fluent: Failed to start span LangGraph: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.


✓ MLflow autologging enabled


2025/11/08 22:45:00 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run adorable-goat-953 at: https://adb-3253299566947192.12.azuredatabricks.net/ml/experiments/897801252548174/runs/a8dd041ca3b14042ac50467dbf51b15b
🧪 View experiment at: https://adb-3253299566947192.12.azuredatabricks.net/ml/experiments/897801252548174
Model logged: models:/m-917a987360744911bce572bc7e4c090c
Model logged: models:/m-917a987360744911bce572bc7e4c090c


# COMMAND ----------

## Step 5: Pre-deployment Validation

Validate the model before deploying.

In [9]:
# COMMAND ----------

mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/agent",
    input_data={"input": [{"role": "user", "content": "What is 7*6 in Python?"}]},
    env_manager="uv",
)


2025/11/08 22:45:05 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2025/11/08 22:45:05 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'

2025/11/08 22:45:08 INFO mlflow.utils.virtualenv: Environment /tmp/virtualenv_envs/mlflow-fe38911592d12d9b07553a1cb2d44173ca6747d4 already exists
2025/11/08 22:45:08 WARNING mlflow.utils.databricks_utils: Missing required environment variable `SPARK_LOCAL_REMOTE` or `SPARK_REMOTE`. These are necessary to initialize the WorkspaceClient with serverless compute in a subprocess in Databricks for UC function execution. Setting the value to 'true'.
2025/11/08 22:45:08 INFO mlflow.utils.environment: === Running command '['bash', '-c', 'source /tmp/virtualenv_envs/mlflow-fe38911592d12d9b07553a1cb2d44173ca6747d4/bin/activate && python -c ""']'

2025/11/08 22:45:08 INFO mlflow.utils.virtualenv: Environment /tmp/virtualenv_envs/mlflow-fe38911592d12d9b07553a1cb2d44173ca6747d4 alr

✓ MLflow autologging enabled
{"object": "response", "output": [{"type": "message", "id": "lc_run--80ee1d12-6c02-42ec-97c8-0bc4423ecade", "content": [{"text": "I can calculate 7*6 using Python. Let me run the code for you:", "type": "output_text"}], "role": "assistant"}, {"type": "function_call", "id": "lc_run--80ee1d12-6c02-42ec-97c8-0bc4423ecade", "call_id": "toolu_bdrk_01UYADGEiB1ForPtNhJ3fcZN", "name": "system__ai__python_exec", "arguments": "{\"code\": \"print(7 * 6)\"}"}, {"type": "function_call_output", "call_id": "toolu_bdrk_01UYADGEiB1ForPtNhJ3fcZN", "output": "{\"is_truncated\":false,\"columns\":[\"output\"],\"rows\":[[\"42\\n\"]]}"}, {"type": "message", "id": "lc_run--24ace0f4-336c-4482-8d1d-45c8acbc0c0d", "content": [{"text": "The result of 7*6 in Python is 42.", "type": "output_text"}], "role": "assistant"}]}{"object": "response", "output": [{"type": "message", "id": "lc_run--80ee1d12-6c02-42ec-97c8-0bc4423ecade", "content": [{"text": "I can calculate 7*6 using Python. Let 

2025/11/08 22:45:15 INFO mlflow.tracing.export.async_export_queue: Flushing the async trace logging queue before program exit. This may take a while...


# COMMAND ----------

## Step 6: Register to Unity Catalog

Register the agent to Unity Catalog before deployment.

In [10]:
# COMMAND ----------

import os

# Set registry URI with profile (same format as tracking URI)
profile_name = os.getenv("DATABRICKS_CONFIG_PROFILE", "development")
mlflow.set_registry_uri(f"databricks-uc://{profile_name}")
print(f"✓ Registry URI set to: databricks-uc://{profile_name}")

# TODO: define the catalog, schema, and model name for your UC model
catalog = "rag"
schema = "development"
model_name = "langgraph_mcp_agent"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# Register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, 
    name=UC_MODEL_NAME
)

print(f"Model registered: {UC_MODEL_NAME} (version {uc_registered_model_info.version})")

✓ Registry URI set to: databricks-uc://development


Registered model 'rag.development.langgraph_mcp_agent' already exists. Creating a new version of this model...

Uploading artifacts: 100%|██████████| 10/10 [00:00<00:00, 11.10it/s]

🔗 Created version '3' of model 'rag.development.langgraph_mcp_agent': https://adb-3253299566947192.12.azuredatabricks.net/explore/data/models/rag/development/langgraph_mcp_agent/version/3
🔗 Created version '3' of model 'rag.development.langgraph_mcp_agent': https://adb-3253299566947192.12.azuredatabricks.net/explore/data/models/rag/development/langgraph_mcp_agent/version/3


Model registered: rag.development.langgraph_mcp_agent (version 3)


# COMMAND ----------

## Step 7: Deploy the Agent

Deploy the agent to a model serving endpoint.

In [ ]:
# COMMAND ----------

from databricks import agents

deployment_info = agents.deploy(
    UC_MODEL_NAME, 
    uc_registered_model_info.version,
    scale_to_zero_enabled=True,
    tags={"endpointSource": "langgraph_mcp_notebook"}
)

print(f"Agent deployed successfully!")
print(f"Endpoint: {deployment_info}")


2025/11/08 22:45:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/08 22:45:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
🔗 View Logged Model at: https://adb-3253299566947192.12.azuredatabricks.net/ml/experiments/897801252548174/models/m-530aa6d09e834f0a8f62df2744791055
🔗 View Logged Model at: https://adb-3253299566947192.12.azuredatabricks.net/ml/experiments/897801252548174/models/m-530aa6d09e834f0a8f62df2744791055
2025/11/08 22:45:25 INFO mlflow.pyfunc: Validating input example against model signature
2025/11/08 22:45:25 INFO mlflow.pyfunc: Validating input example against model signature
Successfully registered model 'rag.development.feedback'.
Successfully registered model 'rag.development.feedback'.
Uploading artifacts: 100%|██████████| 8/8 [00:00<00:00, 19.98it/s]

🔗 Created version '1' of model 'rag.development.feedback': https://adb-3253299566947192.12.azuredatabricks.net/explore/data/m

🏃 View run feedback-model at: https://adb-3253299566947192.12.azuredatabricks.net/ml/experiments/897801252548174/runs/41a7418d740a4bebaf00dfdeb3c1c693
🧪 View experiment at: https://adb-3253299566947192.12.azuredatabricks.net/ml/experiments/897801252548174

    Deployment of rag.development.langgraph_mcp_agent version 3 initiated.  This can take up to 15 minutes and the Review App & Query Endpoint will not work until this deployment finishes.

    View status: https://adb-3253299566947192.12.azuredatabricks.net/ml/endpoints/agents_rag-development-langgraph_mcp_agent/
    Review App: https://adb-3253299566947192.12.azuredatabricks.net/ml/review-v2/34911c7cde294e0bbfafd6c85f583473/chat
    Monitor: https://adb-3253299566947192.12.azuredatabricks.net/ml/experiments/897801252548174?compareRunsMode=TRACES

You can refer back to the links above from the endpoint detail page at https://adb-3253299566947192.12.azuredatabricks.net/ml/endpoints/agents_rag-development-langgraph_mcp_agent/.
Agent d

# COMMAND ----------

## Next Steps

After your agent is deployed, you can:
- Chat with it in AI Playground
- Share it with SMEs for feedback
- Embed it in a production application
- Monitor its performance and traces